In [1]:
from pymongo import MongoClient

# Connecting to MongoDB

In [2]:
client = MongoClient("mongodb://localhost:27017")

# Helper Functions

In [3]:
def print_cursor(c):
    for i in cursor:
        print(i)

# Dataset 1

In [4]:
db = client.universities

In [5]:
client.drop_database("universities")

In [6]:
db.universities.insert_many([
{
  "country" : "Spain",
  "city" : "Salamanca",
  "name" : "USAL",
  "location" : {
    "type" : "Point",
    "coordinates" : [ -5.6722512,17, 40.9607792 ]
  },
  "students" : [
    { "year" : 2014, "number" : 24774 },
    { "year" : 2015, "number" : 23166 },
    { "year" : 2016, "number" : 21913 },
    { "year" : 2017, "number" : 21715 }
  ]
},
{
  "country" : "Spain",
  "city" : "Salamanca",
  "name" : "UPSA",
  "location" : {
    "type" : "Point",
    "coordinates" : [ -5.6691191,17, 40.9631732 ]
  },
  "students" : [
    { "year" : 2014, "number" : 4788 },
    { "year" : 2015, "number" : 4821 },
    { "year" : 2016, "number" : 6550 },
    { "year" : 2017, "number" : 6125 }
  ]
}
])


db.courses.insert_many([
{
  "university" : "USAL",
  "name" : "Computer Science",
  "level" : "Excellent"
},
{
  "university" : "USAL",
  "name" : "Electronics",
  "level" : "Intermediate"
},
{
  "university" : "USAL",
  "name" : "Communication",
  "level" : "Excellent"
}
])

# Aggregation stages

## MongoDB `$match`

In [8]:
cursor = db.universities.aggregate([
  { "$match" : { "country" : "Spain", "city" : "Salamanca" } }
])

print_cursor(cursor)

{'_id': ObjectId('611d18308ba0e401218f6be1'), 'country': 'Spain', 'city': 'Salamanca', 'name': 'USAL', 'location': {'type': 'Point', 'coordinates': [-5.6722512, 17, 40.9607792]}, 'students': [{'year': 2014, 'number': 24774}, {'year': 2015, 'number': 23166}, {'year': 2016, 'number': 21913}, {'year': 2017, 'number': 21715}]}
{'_id': ObjectId('611d18308ba0e401218f6be2'), 'country': 'Spain', 'city': 'Salamanca', 'name': 'UPSA', 'location': {'type': 'Point', 'coordinates': [-5.6691191, 17, 40.9631732]}, 'students': [{'year': 2014, 'number': 4788}, {'year': 2015, 'number': 4821}, {'year': 2016, 'number': 6550}, {'year': 2017, 'number': 6125}]}


## MongoDB `$project`

In [11]:
cursor = db.universities.aggregate([
  { "$project" : { "_id" : 0, "country" : 1, "city" : 1 } }
])

print_cursor(cursor)

{'country': 'Spain', 'city': 'Salamanca'}
{'country': 'Spain', 'city': 'Salamanca'}


## MongoDB `$group`

In [12]:
cursor = db.universities.aggregate([
  { "$group" : { "_id" : "$city", "totaldocs" : { "$sum" : 1 } } }
])

print_cursor(cursor)

{'_id': 'Salamanca', 'totaldocs': 2}


## MongoDB `$out`

In [13]:
db.universities.aggregate([
  { "$group" : { "_id" : "$city", "totaldocs" : { "$sum" : 1 } } },
      { "$out" : "aggResults" } #record in Results 
])

cursor = db.aggResults.find()
print_cursor(cursor)

{'_id': 'Salamanca', 'totaldocs': 2}


## MongoDB `$unwind`

In [14]:
cursor = db.universities.aggregate([
  { "$match" : { "name" : "USAL" } },
  { "$unwind" : "$students" } #extract the array and use the supelements to create new documents with it 
])

print_cursor(cursor)

{'_id': ObjectId('611d18308ba0e401218f6be1'), 'country': 'Spain', 'city': 'Salamanca', 'name': 'USAL', 'location': {'type': 'Point', 'coordinates': [-5.6722512, 17, 40.9607792]}, 'students': {'year': 2014, 'number': 24774}}
{'_id': ObjectId('611d18308ba0e401218f6be1'), 'country': 'Spain', 'city': 'Salamanca', 'name': 'USAL', 'location': {'type': 'Point', 'coordinates': [-5.6722512, 17, 40.9607792]}, 'students': {'year': 2015, 'number': 23166}}
{'_id': ObjectId('611d18308ba0e401218f6be1'), 'country': 'Spain', 'city': 'Salamanca', 'name': 'USAL', 'location': {'type': 'Point', 'coordinates': [-5.6722512, 17, 40.9607792]}, 'students': {'year': 2016, 'number': 21913}}
{'_id': ObjectId('611d18308ba0e401218f6be1'), 'country': 'Spain', 'city': 'Salamanca', 'name': 'USAL', 'location': {'type': 'Point', 'coordinates': [-5.6722512, 17, 40.9607792]}, 'students': {'year': 2017, 'number': 21715}}


## MongoDB `$sort`

In [19]:
cursor = db.universities.aggregate([
  { "$match" : { "city" : "Salamanca" } },
  { "$unwind" : "$students" },
  { "$project" : { "_id" : 0, "name": 1, "students.year" : 1, "students.number" : 1 } },
  { "$sort" : { "students.number" : -1 } }#descending
])

print_cursor(cursor)

{'name': 'USAL', 'students': {'year': 2014, 'number': 24774}}
{'name': 'USAL', 'students': {'year': 2015, 'number': 23166}}
{'name': 'USAL', 'students': {'year': 2016, 'number': 21913}}
{'name': 'USAL', 'students': {'year': 2017, 'number': 21715}}
{'name': 'UPSA', 'students': {'year': 2016, 'number': 6550}}
{'name': 'UPSA', 'students': {'year': 2017, 'number': 6125}}
{'name': 'UPSA', 'students': {'year': 2015, 'number': 4821}}
{'name': 'UPSA', 'students': {'year': 2014, 'number': 4788}}


## MongoDB `$limit`

In [20]:
cursor = db.universities.aggregate([
  { "$match" : { "name" : "USAL" } },
  { "$unwind" : "$students" },
  { "$project" : { "_id" : 0, "students.year" : 1, "students.number" : 1 } },
  { "$sort" : { "students.number" : -1 } },
  { "$limit" : 3 }
])

print_cursor(cursor)

{'students': {'year': 2014, 'number': 24774}}
{'students': {'year': 2015, 'number': 23166}}
{'students': {'year': 2016, 'number': 21913}}


## MongoDB `$addFields`

In [21]:
# cursor = db.universities.aggregate([
#   { "$match" : { "name" : "USAL" } },
#   { "$addFields" : { "foundation_year" : 1218 } }
# ])
cursor = db.universities.aggregate([
  { "$match" : { "name" : "UPSA" } },
  { "$addFields" : { "foundation_year" : 1220 } }
])

print_cursor(cursor)

{'_id': ObjectId('611d18308ba0e401218f6be2'), 'country': 'Spain', 'city': 'Salamanca', 'name': 'UPSA', 'location': {'type': 'Point', 'coordinates': [-5.6691191, 17, 40.9631732]}, 'students': [{'year': 2014, 'number': 4788}, {'year': 2015, 'number': 4821}, {'year': 2016, 'number': 6550}, {'year': 2017, 'number': 6125}], 'foundation_year': 1220}


## MongoDB `$count`

In [22]:
cursor = db.universities.aggregate([
  { "$match" : { "name" : "USAL" } },
  { "$unwind" : "$students" },
  { "$count" : "total_documents" }
])

print_cursor(cursor)

{'total_documents': 4}


## MongoDB `$lookup`

If you want this query to run fast, you are going to need to index the `name` field in the `universities` collection and the `university` field in the `courses` collection.

In [28]:
cursor = db.universities.aggregate([
  { "$match" : { "name" : "USAL" } },
  { "$project" : { "_id" : 0, "name" : 1 } },
  { "$lookup" : {
      "from" : "courses",
      "localField" : "name",
      "foreignField" : "university",
      "as" : "courses"
  } },
  { "$project" : { "_id" : 0, "courses.university": 1, "courses.name" : 1  } }
])

print_cursor(cursor)

{'courses': [{'university': 'USAL', 'name': 'Computer Science'}, {'university': 'USAL', 'name': 'Electronics'}, {'university': 'USAL', 'name': 'Communication'}]}


## MongoDB `$sortByCount`

This stage is a shortcut for grouping, counting and then sorting in descending order the number of different values in a field.

In [29]:
# cursor = db.courses.aggregate([
#   { "$sortByCount" : "$level" }
# ])
cursor = db.universities.aggregate([
  { "$sortByCount" : "$country" }
])

print_cursor(cursor)

{'_id': 'Spain', 'count': 2}


## MongoDB `$facet`

Sometimes when creating a report on data, you find that you need to do the same preliminary processing for a number of reports, and you are faced with having to create and maintain an intermediate collection.

In [30]:
cursor = db.universities.aggregate([
  { "$match" : { "name" : "USAL" } },
  { "$lookup" : {
      "from" : "courses",
      "localField" : "name",
      "foreignField" : "university",
      "as" : "courses"
  } }
])

print_cursor(cursor)

{'_id': ObjectId('611d18308ba0e401218f6be1'), 'country': 'Spain', 'city': 'Salamanca', 'name': 'USAL', 'location': {'type': 'Point', 'coordinates': [-5.6722512, 17, 40.9607792]}, 'students': [{'year': 2014, 'number': 24774}, {'year': 2015, 'number': 23166}, {'year': 2016, 'number': 21913}, {'year': 2017, 'number': 21715}], 'courses': [{'_id': ObjectId('611d18308ba0e401218f6be3'), 'university': 'USAL', 'name': 'Computer Science', 'level': 'Excellent'}, {'_id': ObjectId('611d18308ba0e401218f6be4'), 'university': 'USAL', 'name': 'Electronics', 'level': 'Intermediate'}, {'_id': ObjectId('611d18308ba0e401218f6be5'), 'university': 'USAL', 'name': 'Communication', 'level': 'Excellent'}]}


In [33]:
cursor = db.universities.aggregate([
  { "$match" : { "name" : "USAL" } },
  { "$lookup" : {
      "from" : "courses",
      "localField" : "name",
      "foreignField" : "university",
      "as" : "courses"
  } },
         { "$unwind" : "$courses" },
#          { "$sortByCount" : "$courses.level" }
        { "$sortByCount" : "$courses.name" }
])

print_cursor(cursor)

{'_id': 'Communication', 'count': 1}
{'_id': 'Electronics', 'count': 1}
{'_id': 'Computer Science', 'count': 1}


In [34]:
cursor = db.universities.aggregate([
  { "$match" : { "name" : "USAL" } },
  { "$lookup" : {
      "from" : "courses",
      "localField" : "name",
      "foreignField" : "university",
      "as" : "courses"
  } },
         { "$unwind" : "$students" },
         { "$project" : { "_id" : 0, "students" : 1 } },
         { "$sort" : { "students.number" : 1 } }, # ascending
         { "$limit" : 1 }
])

print_cursor(cursor)

{'students': {'year': 2017, 'number': 21715}}


In [35]:
cursor = db.universities.aggregate([
  { "$match" : { "name" : "USAL" } },
  { "$lookup" : {
      "from" : "courses",
      "localField" : "name",
      "foreignField" : "university",
      "as" : "courses"
  } },
  { "$facet" : {
      "countingLevels" :
      [
         { "$unwind" : "$courses" },
         { "$sortByCount" : "$courses.level" }
#          { "$sortByCount" : "$courses.name" }
      ],
      "yearWithLessStudents" :
      [
         { "$unwind" : "$students" },
         { "$project" : { "_id" : 0, "students" : 1 } },
         { "$sort" : { "students.number" : 1 } },
         { "$limit" : 1 }
      ]
  } }
])

print_cursor(cursor)

{'countingLevels': [{'_id': 'Excellent', 'count': 2}, {'_id': 'Intermediate', 'count': 1}], 'yearWithLessStudents': [{'students': {'year': 2017, 'number': 21715}}]}


# Dataset 2

Use `mongoimport` to load http://media.mongodb.org/zips.json into your mongod instance.


In [37]:
db = client.zips

In [38]:
cursor = db.zipcodes.find({"_id" : "47906"})
print_cursor(cursor)

{'_id': '47906', 'city': 'WEST LAFAYETTE', 'loc': [-86.923661, 40.444025], 'pop': 54702, 'state': 'IN'}


# Aggregation Pipeline

### Return States with Populations above 10 Million

`SELECT state, SUM(pop) AS totalPop
FROM zipcodes
GROUP BY state
HAVING totalPop >= (10*1000*1000)`

In [40]:
cursor = db.zipcodes.aggregate( [
   { "$group": { "_id": "$state", "totalPop": { "$sum": "$pop" } } },
#    { "$match": { "totalPop": { "$gte": 10*1000*1000 } } } #greater
   { "$match": { "totalPop": { "$lte": 10*1000*1000 } } } #less
] )

print_cursor(cursor)

{'_id': 'SD', 'totalPop': 695397}
{'_id': 'AR', 'totalPop': 2350725}
{'_id': 'WA', 'totalPop': 4866692}
{'_id': 'MS', 'totalPop': 2573216}
{'_id': 'AK', 'totalPop': 544698}
{'_id': 'OK', 'totalPop': 3145585}
{'_id': 'VT', 'totalPop': 562758}
{'_id': 'GA', 'totalPop': 6478216}
{'_id': 'AL', 'totalPop': 4040587}
{'_id': 'WI', 'totalPop': 4891769}
{'_id': 'HI', 'totalPop': 1108229}
{'_id': 'MT', 'totalPop': 798948}
{'_id': 'UT', 'totalPop': 1722850}
{'_id': 'TN', 'totalPop': 4876457}
{'_id': 'SC', 'totalPop': 3486703}
{'_id': 'DC', 'totalPop': 606900}
{'_id': 'MI', 'totalPop': 9295297}
{'_id': 'DE', 'totalPop': 666168}
{'_id': 'ID', 'totalPop': 1006749}
{'_id': 'IA', 'totalPop': 2776420}
{'_id': 'KY', 'totalPop': 3675484}
{'_id': 'RI', 'totalPop': 1003218}
{'_id': 'MD', 'totalPop': 4781379}
{'_id': 'LA', 'totalPop': 4217595}
{'_id': 'CO', 'totalPop': 3293755}
{'_id': 'VA', 'totalPop': 6181479}
{'_id': 'NM', 'totalPop': 1515069}
{'_id': 'OR', 'totalPop': 2842321}
{'_id': 'CT', 'totalPop': 

### Return Average City Population by State

In [44]:
cursor = db.zipcodes.aggregate( [
   { "$group": { "_id": { "state": "$state", "city": "$city" }, "pop": { "$sum": "$pop" } } },
   { "$sort": { "_id.state": 1 } }
] )

print_cursor(cursor)

{'_id': {'state': 'AK', 'city': 'PORT HEIDEN'}, 'pop': 119}
{'_id': {'state': 'AK', 'city': 'RUSSIAN MISSION'}, 'pop': 0}
{'_id': {'state': 'AK', 'city': 'POINT HOPE'}, 'pop': 640}
{'_id': {'state': 'AK', 'city': 'ELMENDORF AFB'}, 'pop': 7907}
{'_id': {'state': 'AK', 'city': 'COLDFOOT'}, 'pop': 19316}
{'_id': {'state': 'AK', 'city': 'CRAIG'}, 'pop': 1398}
{'_id': {'state': 'AK', 'city': 'CLAM GULCH'}, 'pop': 133}
{'_id': {'state': 'AK', 'city': 'KWETHLUK'}, 'pop': 558}
{'_id': {'state': 'AK', 'city': 'PEDRO BAY'}, 'pop': 59}
{'_id': {'state': 'AK', 'city': 'KALTAG'}, 'pop': 240}
{'_id': {'state': 'AK', 'city': 'KAKTOVIK'}, 'pop': 245}
{'_id': {'state': 'AK', 'city': 'SHUNGNAK'}, 'pop': 0}
{'_id': {'state': 'AK', 'city': 'MC GRATH'}, 'pop': 618}
{'_id': {'state': 'AK', 'city': 'POINT LAY'}, 'pop': 139}
{'_id': {'state': 'AK', 'city': 'SAVOONGA'}, 'pop': 519}
{'_id': {'state': 'AK', 'city': 'AMBLER'}, 'pop': 8}
{'_id': {'state': 'AK', 'city': 'NORTH POLE'}, 'pop': 14672}
{'_id': {'state'

{'_id': {'state': 'AL', 'city': 'SAWYERVILLE'}, 'pop': 1684}
{'_id': {'state': 'AL', 'city': 'GRAHAM'}, 'pop': 374}
{'_id': {'state': 'AL', 'city': 'BILLINGSLEY'}, 'pop': 1869}
{'_id': {'state': 'AL', 'city': 'WEBB'}, 'pop': 1810}
{'_id': {'state': 'AL', 'city': 'SKIPPERVILLE'}, 'pop': 1048}
{'_id': {'state': 'AL', 'city': 'CHANCELLOR'}, 'pop': 620}
{'_id': {'state': 'AL', 'city': 'ESTILLFORK'}, 'pop': 718}
{'_id': {'state': 'AL', 'city': 'CRAGFORD'}, 'pop': 796}
{'_id': {'state': 'AL', 'city': 'CODEN'}, 'pop': 3897}
{'_id': {'state': 'AL', 'city': 'ONEONTA'}, 'pop': 8956}
{'_id': {'state': 'AL', 'city': 'COLUMBIANA'}, 'pop': 4486}
{'_id': {'state': 'AL', 'city': 'ELKMONT'}, 'pop': 8013}
{'_id': {'state': 'AL', 'city': 'GUNTERSVILLE'}, 'pop': 11234}
{'_id': {'state': 'AL', 'city': 'PEROTE'}, 'pop': 1792}
{'_id': {'state': 'AL', 'city': 'GARDENDALE'}, 'pop': 17968}
{'_id': {'state': 'AL', 'city': 'GLENWOOD'}, 'pop': 109}
{'_id': {'state': 'AL', 'city': 'FORKLAND'}, 'pop': 2121}
{'_id': 

{'_id': {'state': 'CA', 'city': 'POINT REYES STAT'}, 'pop': 951}
{'_id': {'state': 'CA', 'city': 'MEADOW VISTA'}, 'pop': 3314}
{'_id': {'state': 'CA', 'city': 'ALISO VIEJO'}, 'pop': 16455}
{'_id': {'state': 'CA', 'city': 'SAN ARDO'}, 'pop': 1684}
{'_id': {'state': 'CA', 'city': 'WINTERS'}, 'pop': 6253}
{'_id': {'state': 'CA', 'city': 'MC KINLEYVILLE'}, 'pop': 32283}
{'_id': {'state': 'CA', 'city': 'KORBEL'}, 'pop': 187}
{'_id': {'state': 'CA', 'city': 'MCCLELLAN AFB'}, 'pop': 541}
{'_id': {'state': 'CA', 'city': 'WILLIAMS'}, 'pop': 3094}
{'_id': {'state': 'CA', 'city': 'BONITA'}, 'pop': 16579}
{'_id': {'state': 'CA', 'city': 'IMPERIAL'}, 'pop': 6092}
{'_id': {'state': 'CA', 'city': 'APPLEGATE'}, 'pop': 1898}
{'_id': {'state': 'CA', 'city': 'LE GRAND'}, 'pop': 1810}
{'_id': {'state': 'CA', 'city': 'CAMARILLO'}, 'pop': 63662}
{'_id': {'state': 'CA', 'city': 'LUDLOW'}, 'pop': 383}
{'_id': {'state': 'CA', 'city': 'CHICO'}, 'pop': 27452}
{'_id': {'state': 'CA', 'city': 'EL TORO MARINE C'}, 

{'_id': {'state': 'FL', 'city': 'SOPCHOPPY'}, 'pop': 3335}
{'_id': {'state': 'FL', 'city': 'VENICE'}, 'pop': 9069}
{'_id': {'state': 'FL', 'city': 'CHATTAHOOCHEE'}, 'pop': 6325}
{'_id': {'state': 'FL', 'city': 'SANTA FE'}, 'pop': 9414}
{'_id': {'state': 'FL', 'city': 'FOUNTAIN'}, 'pop': 1869}
{'_id': {'state': 'FL', 'city': 'WALDO'}, 'pop': 1676}
{'_id': {'state': 'FL', 'city': 'CHRISTMAS'}, 'pop': 2331}
{'_id': {'state': 'FL', 'city': 'BALDWIN'}, 'pop': 5830}
{'_id': {'state': 'FL', 'city': 'CARROLLWOOD'}, 'pop': 59845}
{'_id': {'state': 'FL', 'city': 'GRENELEFE'}, 'pop': 23835}
{'_id': {'state': 'FL', 'city': 'PORT SAINT JOHN'}, 'pop': 17351}
{'_id': {'state': 'FL', 'city': 'LORIDA'}, 'pop': 1186}
{'_id': {'state': 'FL', 'city': 'FRUITLAND PARK'}, 'pop': 8513}
{'_id': {'state': 'FL', 'city': 'DAVENPORT'}, 'pop': 8268}
{'_id': {'state': 'FL', 'city': 'COCOA BEACH'}, 'pop': 14989}
{'_id': {'state': 'FL', 'city': 'NEW SMYRNA BEACH'}, 'pop': 26793}
{'_id': {'state': 'FL', 'city': 'VERO B

{'_id': {'state': 'IA', 'city': 'WEST CHESTER'}, 'pop': 500}
{'_id': {'state': 'IA', 'city': 'LITTLE CEDAR'}, 'pop': 248}
{'_id': {'state': 'IA', 'city': 'UNION'}, 'pop': 937}
{'_id': {'state': 'IA', 'city': 'VOLGA'}, 'pop': 562}
{'_id': {'state': 'IA', 'city': 'RUTLAND'}, 'pop': 529}
{'_id': {'state': 'IA', 'city': 'MODALE'}, 'pop': 505}
{'_id': {'state': 'IA', 'city': 'NEW LONDON'}, 'pop': 3627}
{'_id': {'state': 'IA', 'city': 'OLLIE'}, 'pop': 631}
{'_id': {'state': 'IA', 'city': 'MONTROSE'}, 'pop': 1857}
{'_id': {'state': 'IA', 'city': 'BUFFALO CENTER'}, 'pop': 1566}
{'_id': {'state': 'IA', 'city': 'RAYMOND'}, 'pop': 367}
{'_id': {'state': 'IA', 'city': 'PROMISE CITY'}, 'pop': 370}
{'_id': {'state': 'IA', 'city': 'DURANGO'}, 'pop': 874}
{'_id': {'state': 'IA', 'city': 'CAMANCHE'}, 'pop': 5013}
{'_id': {'state': 'IA', 'city': 'PILOT MOUND'}, 'pop': 380}
{'_id': {'state': 'IA', 'city': 'PRINCETON'}, 'pop': 1529}
{'_id': {'state': 'IA', 'city': 'FLORIS'}, 'pop': 1087}
{'_id': {'state':

...

{'_id': {'state': 'SC', 'city': 'TILLMAN'}, 'pop': 1679}
{'_id': {'state': 'SC', 'city': 'SPRINGFIELD'}, 'pop': 2184}
{'_id': {'state': 'SC', 'city': 'MOUNT CARMEL'}, 'pop': 1354}
{'_id': {'state': 'SC', 'city': 'BLACKSTOCK'}, 'pop': 249}
{'_id': {'state': 'SC', 'city': 'HOPKINS'}, 'pop': 12297}
{'_id': {'state': 'SC', 'city': 'PAXVILLE'}, 'pop': 14407}
{'_id': {'state': 'SC', 'city': 'ROCK HILL'}, 'pop': 71993}
{'_id': {'state': 'SC', 'city': 'WARD'}, 'pop': 823}
{'_id': {'state': 'SC', 'city': 'WALLACE'}, 'pop': 3398}
{'_id': {'state': 'SC', 'city': 'FLORENCE'}, 'pop': 68430}
{'_id': {'state': 'SC', 'city': 'ORANGEBURG'}, 'pop': 43095}
{'_id': {'state': 'SC', 'city': 'MC COLL'}, 'pop': 5224}
{'_id': {'state': 'SC', 'city': 'RIDGEVILLE'}, 'pop': 7494}
{'_id': {'state': 'SC', 'city': 'SOCIETY HILL'}, 'pop': 3278}
{'_id': {'state': 'SC', 'city': 'RICHBURG'}, 'pop': 2393}
{'_id': {'state': 'SC', 'city': 'HARLEYVILLE'}, 'pop': 3289}
{'_id': {'state': 'SC', 'city': 'IVA'}, 'pop': 5342}
{'_

{'_id': {'state': 'TX', 'city': 'KLONDIKE'}, 'pop': 729}
{'_id': {'state': 'TX', 'city': 'LOLITA'}, 'pop': 2632}
{'_id': {'state': 'TX', 'city': 'HAWLEY'}, 'pop': 4405}
{'_id': {'state': 'TX', 'city': 'LAWN'}, 'pop': 619}
{'_id': {'state': 'TX', 'city': 'SPICEWOOD'}, 'pop': 2235}
{'_id': {'state': 'TX', 'city': 'DENTON'}, 'pop': 76473}
{'_id': {'state': 'TX', 'city': 'CHILTON'}, 'pop': 2304}
{'_id': {'state': 'TX', 'city': 'MIDLAND'}, 'pop': 106611}
{'_id': {'state': 'TX', 'city': 'CAMERON'}, 'pop': 6965}
{'_id': {'state': 'TX', 'city': 'CARTHAGE'}, 'pop': 11062}
{'_id': {'state': 'TX', 'city': 'PINEHURST'}, 'pop': 2498}
{'_id': {'state': 'TX', 'city': 'MELVIN'}, 'pop': 361}
{'_id': {'state': 'TX', 'city': 'ROXTON'}, 'pop': 975}
{'_id': {'state': 'TX', 'city': 'HUTCHINS'}, 'pop': 2716}
{'_id': {'state': 'TX', 'city': 'COOKVILLE'}, 'pop': 2050}
{'_id': {'state': 'TX', 'city': 'TWITTY'}, 'pop': 2927}
{'_id': {'state': 'TX', 'city': 'WEATHERFORD'}, 'pop': 32439}
{'_id': {'state': 'TX', 'c

{'_id': {'state': 'VA', 'city': 'CONICVILLE'}, 'pop': 4601}
{'_id': {'state': 'VA', 'city': 'HALIFAX'}, 'pop': 7127}
{'_id': {'state': 'VA', 'city': 'NATURAL BRIDGE S'}, 'pop': 1419}
{'_id': {'state': 'VA', 'city': 'SAINT PAUL'}, 'pop': 2680}
{'_id': {'state': 'VA', 'city': 'TIMBERLAKE'}, 'pop': 28755}
{'_id': {'state': 'VA', 'city': 'WATERFORD'}, 'pop': 264}
{'_id': {'state': 'VA', 'city': 'COVESVILLE'}, 'pop': 304}
{'_id': {'state': 'VA', 'city': 'VESUVIUS'}, 'pop': 548}
{'_id': {'state': 'VA', 'city': 'UNIVERSITY'}, 'pop': 25301}
{'_id': {'state': 'VA', 'city': 'NICKELSVILLE'}, 'pop': 2749}
{'_id': {'state': 'VA', 'city': 'CRIMORA'}, 'pop': 2153}
{'_id': {'state': 'VA', 'city': 'GASBURG'}, 'pop': 771}
{'_id': {'state': 'VA', 'city': 'JENKINS BRIDGE'}, 'pop': 301}
{'_id': {'state': 'VA', 'city': 'WHITETOP'}, 'pop': 642}
{'_id': {'state': 'VA', 'city': 'CLIFTON'}, 'pop': 9554}
{'_id': {'state': 'VA', 'city': 'WISE'}, 'pop': 8957}
{'_id': {'state': 'VA', 'city': 'CARRSVILLE'}, 'pop': 1

{'_id': {'state': 'WI', 'city': 'NEW LONDON'}, 'pop': 13037}
{'_id': {'state': 'WI', 'city': 'ASHWAUBENON'}, 'pop': 31339}
{'_id': {'state': 'WI', 'city': 'GAYS MILLS'}, 'pop': 2577}
{'_id': {'state': 'WI', 'city': 'AMBERG'}, 'pop': 917}
{'_id': {'state': 'WI', 'city': 'WASHBURN'}, 'pop': 3177}
{'_id': {'state': 'WI', 'city': 'MERRILL'}, 'pop': 17891}
{'_id': {'state': 'WI', 'city': 'ROBERTS'}, 'pop': 2063}
{'_id': {'state': 'WI', 'city': 'RIDGEWAY'}, 'pop': 924}
{'_id': {'state': 'WI', 'city': 'LA CROSSE'}, 'pop': 63640}
{'_id': {'state': 'WI', 'city': 'ROCKLAND'}, 'pop': 1486}
{'_id': {'state': 'WI', 'city': 'SHEBOYGAN FALLS'}, 'pop': 9457}
{'_id': {'state': 'WI', 'city': 'NEW FRANKEN'}, 'pop': 2640}
{'_id': {'state': 'WI', 'city': 'GREENVILLE'}, 'pop': 1722}
{'_id': {'state': 'WI', 'city': 'WAUMANDEE'}, 'pop': 2271}
{'_id': {'state': 'WI', 'city': 'REESEVILLE'}, 'pop': 2027}
{'_id': {'state': 'WI', 'city': 'NEOPIT'}, 'pop': 14}
{'_id': {'state': 'WI', 'city': 'KENDALL'}, 'pop': 1848

{'_id': {'state': 'WY', 'city': 'FORT BRIDGER'}, 'pop': 3777}
{'_id': {'state': 'WY', 'city': 'OSAGE'}, 'pop': 292}
{'_id': {'state': 'WY', 'city': 'ALCOVA'}, 'pop': 10}
{'_id': {'state': 'WY', 'city': 'HARTVILLE'}, 'pop': 108}
{'_id': {'state': 'WY', 'city': 'TORRINGTON'}, 'pop': 9575}
{'_id': {'state': 'WY', 'city': 'SHELL'}, 'pop': 355}
{'_id': {'state': 'WY', 'city': 'KELLY'}, 'pop': 203}
{'_id': {'state': 'WY', 'city': 'LOVELL'}, 'pop': 4322}
{'_id': {'state': 'WY', 'city': 'SINCLAIR'}, 'pop': 530}
{'_id': {'state': 'WY', 'city': 'STORY'}, 'pop': 63}
{'_id': {'state': 'WY', 'city': 'SHERIDAN'}, 'pop': 20025}
{'_id': {'state': 'WY', 'city': 'PARKMAN'}, 'pop': 148}
{'_id': {'state': 'WY', 'city': 'MEDICINE BOW'}, 'pop': 409}
{'_id': {'state': 'WY', 'city': 'LOST SPRINGS'}, 'pop': 6}
{'_id': {'state': 'WY', 'city': 'DIXON'}, 'pop': 253}
{'_id': {'state': 'WY', 'city': 'BAIROIL'}, 'pop': 236}
{'_id': {'state': 'WY', 'city': 'DUBOIS'}, 'pop': 1493}
{'_id': {'state': 'WY', 'city': 'PINE

In [41]:
cursor = db.zipcodes.aggregate( [
   { "$group": { "_id": { "state": "$state", "city": "$city" }, "pop": { "$sum": "$pop" } } },
   { "$group": { "_id": "$_id.state", "avgCityPop": { "$avg": "$pop" } } }
] )

print_cursor(cursor)

{'_id': 'NE', 'avgCityPop': 3034.882692307692}
{'_id': 'MI', 'avgCityPop': 12087.512353706112}
{'_id': 'CO', 'avgCityPop': 9981.075757575758}
{'_id': 'SC', 'avgCityPop': 11139.626198083068}
{'_id': 'TN', 'avgCityPop': 9656.350495049504}
{'_id': 'IL', 'avgCityPop': 9954.334494773519}
{'_id': 'OH', 'avgCityPop': 12700.839578454332}
{'_id': 'ID', 'avgCityPop': 4320.811158798283}
{'_id': 'DE', 'avgCityPop': 14481.91304347826}
{'_id': 'KY', 'avgCityPop': 4767.164721141375}
{'_id': 'DC', 'avgCityPop': 303450.0}
{'_id': 'IA', 'avgCityPop': 3123.0821147356583}
{'_id': 'OK', 'avgCityPop': 6155.743639921722}
{'_id': 'GA', 'avgCityPop': 11547.62210338681}
{'_id': 'AL', 'avgCityPop': 7907.2152641878665}
{'_id': 'SD', 'avgCityPop': 1839.6746031746031}
{'_id': 'VT', 'avgCityPop': 2315.8765432098767}
{'_id': 'WA', 'avgCityPop': 12258.670025188916}
{'_id': 'MS', 'avgCityPop': 7524.023391812865}
{'_id': 'MT', 'avgCityPop': 2593.987012987013}
{'_id': 'UT', 'avgCityPop': 9518.508287292818}
{'_id': 'HI', 

### Return Largest and Smallest Cities by State

In [45]:
cursor = db.zipcodes.aggregate( [
   { "$group":
      {
        "_id": { "state": "$state", "city": "$city" },
        "pop": { "$sum": "$pop" }
      }
   },
   { "$sort": { "pop": 1 } },
   { "$group":
      {
        "_id" : "$_id.state",
        "biggestCity":  { "$last": "$_id.city" },
        "biggestPop":   { "$last": "$pop" },
        "smallestCity": { "$first": "$_id.city" },
        "smallestPop":  { "$first": "$pop" }
      }
   },
  { "$project":
    { "_id": 0,
      "state": "$_id",
      "biggestCity":  { "name": "$biggestCity",  "pop": "$biggestPop" },
      "smallestCity": { "name": "$smallestCity", "pop": "$smallestPop" }
    }
  }
] )

print_cursor(cursor)

{'biggestCity': {'name': 'PHOENIX', 'pop': 890853}, 'smallestCity': {'name': 'HUALAPAI', 'pop': 2}, 'state': 'AZ'}
{'biggestCity': {'name': 'SEATTLE', 'pop': 520096}, 'smallestCity': {'name': 'BENGE', 'pop': 2}, 'state': 'WA'}
{'biggestCity': {'name': 'SIOUX FALLS', 'pop': 102046}, 'smallestCity': {'name': 'ZEONA', 'pop': 8}, 'state': 'SD'}
{'biggestCity': {'name': 'JACKSON', 'pop': 204788}, 'smallestCity': {'name': 'CHUNKY', 'pop': 79}, 'state': 'MS'}
{'biggestCity': {'name': 'HONOLULU', 'pop': 396643}, 'smallestCity': {'name': 'NINOLE', 'pop': 0}, 'state': 'HI'}
{'biggestCity': {'name': 'TULSA', 'pop': 389072}, 'smallestCity': {'name': 'SOUTHARD', 'pop': 8}, 'state': 'OK'}
{'biggestCity': {'name': 'INDIANAPOLIS', 'pop': 348868}, 'smallestCity': {'name': 'WESTPOINT', 'pop': 145}, 'state': 'IN'}
{'biggestCity': {'name': 'BURLINGTON', 'pop': 39127}, 'smallestCity': {'name': 'UNIV OF VERMONT', 'pop': 0}, 'state': 'VT'}
{'biggestCity': {'name': 'BIRMINGHAM', 'pop': 242606}, 'smallestCity'